In [18]:
import pickle

# Load the augmented images and labels from the file
with open('../local_data/augmented_data.pickle', 'rb') as f:
    augmented_images, augmented_labels = pickle.load(f)

In [20]:
import tensorflow as tf
import numpy as np
from keras.utils import to_categorical

augmented_images = np.array(augmented_images, dtype=np.float32)
augmented_labels = np.array(augmented_labels, dtype=np.int32)
augmented_labels = to_categorical(augmented_labels)

from sklearn.model_selection import train_test_split

X_train, X_val, y_train, y_val = train_test_split(augmented_images, augmented_labels, test_size=0.2, random_state=42)

In [40]:
from tensorflow.keras.applications import MobileNetV2
from tensorflow.keras.layers import GlobalAveragePooling2D, Dense, Dropout, Conv2D, MaxPooling2D, Normalization, Flatten
from tensorflow.keras.models import Model

base_model = tf.keras.applications.MobileNetV2(input_shape=(512, 512, 3), include_top=False, weights='imagenet')

base_model.trainable = False

x = base_model.output
x = Conv2D(512, (3, 3), activation='relu')(x)
x = MaxPooling2D((2, 2))(x)
x = Normalization()(x)

x = Conv2D(512, (3, 3), activation='relu')(x)
x = MaxPooling2D((2, 2))(x)
x = Normalization()(x)
x = Flatten()(x)

x = Dense(512, activation='relu')(x)
x = Dropout(0.5)(x)
predictions = Dense(2, activation='softmax')(x)

model = Model(inputs=base_model.input, outputs=predictions)

C:\Users\juliu\AppData\Local\Temp\ipykernel_15440\57540098.py:5: UserWarning: `input_shape` is undefined or non-square, or `rows` is not in [96, 128, 160, 192, 224]. Weights for input shape (224, 224) will be loaded as the default.
  base_model = tf.keras.applications.MobileNetV2(input_shape=(512, 512, 3), include_top=False, weights='imagenet')


In [41]:
model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])

In [42]:
model.fit(X_train, y_train, epochs=5, validation_data=(X_val, y_val))

Epoch 1/5
50/50 ━━━━━━━━━━━━━━━━━━━━ 96s 2s/step - accuracy: 0.6601 - loss: 3.7399 - val_accuracy: 0.9450 - val_loss: 0.1367
Epoch 2/5
50/50 ━━━━━━━━━━━━━━━━━━━━ 91s 2s/step - accuracy: 0.9923 - loss: 0.0252 - val_accuracy: 1.0000 - val_loss: 6.0398e-04
Epoch 3/5
50/50 ━━━━━━━━━━━━━━━━━━━━ 92s 2s/step - accuracy: 0.9982 - loss: 0.0052 - val_accuracy: 0.9925 - val_loss: 0.0323
Epoch 4/5
50/50 ━━━━━━━━━━━━━━━━━━━━ 90s 2s/step - accuracy: 0.9941 - loss: 0.0188 - val_accuracy: 0.9975 - val_loss: 0.0047
Epoch 5/5
50/50 ━━━━━━━━━━━━━━━━━━━━ 90s 2s/step - accuracy: 0.9993 - loss: 0.0032 - val_accuracy: 1.0000 - val_loss: 0.0022


In [74]:
model.save('../models/pre_model_green_5.h5')